In [1]:
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("yolo11m-obb.pt")  # load a pretrained model (recommended for training)


results = model.train(
    # task = 'OBB',
    data = "data.yaml",
    epochs = 50,
    batch = 16,
    imgsz = 256,
    name='ship_detection',  # 저장될 모델의 이름
    device=0,           # 사용할 GPU 장치 (CPU의 경우 device='cpu')
    single_cls=True,    # 단일 클래스 학습 여부
    visualize=True,     # 학습과정에서 결과 이미지 시각화 여부
)

100%|██████████| 40.5M/40.5M [00:03<00:00, 13.3MB/s]


New https://pypi.org/project/ultralytics/8.3.5 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.4  Python-3.11.4 torch-2.0.1 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=obb, mode=train, model=yolo11m-obb.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=ship_detection2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=True, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fa

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pook0612 (limbw). Use `wandb login --relogin` to force relogin


Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...
AMP: checks passed 


train: Scanning D:\Workspace\SatellieteIMGDetection\datasets\labels\train... 357 images, 0 backgrounds, 4 corrupt: 100%|██████████| 357/357 [00:00<00:00, 385.95it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      25/50      2.03G        nan        nan        nan          7        256: 100%|██████████| 23/23 [00:19<00:00,  1.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.35it/s]

                   all         37       1653          0          0          0          0



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      26/50      2.06G        nan        nan        nan          1        256: 100%|██████████| 23/23 [00:19<00:00,  1.17it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/3 [00:00<?, ?it/s]